In [1]:
from sgp4.api import Satrec
from sgp4.api import jday

# First, an example with one TLE

First we take an example of a TLE on the celestrack website. 

For instance we can take the latest TLE of the ISS-ZARYA at the following link. 

https://celestrak.com/satcat/search-results.php

Click on the TLE icon in the column "Latest Data".

You will see something like that :


**ISS (ZARYA)**            

**1 25544U 98067A   21035.51324206  .00001077  00000-0  27754-4 0  9998**

**2 25544  51.6455 278.9410 0002184 336.6191  80.6984 15.48940116268036**


We then retrieve the content of the TLE in the variable s (first row) & t (second row).

In [2]:
s = '1 25544U 98067A   21035.51324206  .00001077  00000-0  27754-4 0  9998'

t = '2 25544  51.6455 278.9410 0002184 336.6191  80.6984 15.48940116268036'

satellite = Satrec.twoline2rv(s, t)

We pick a date & associate it with the variables jd (Julian Date) and fr (fraction of Julian Date).

We give the jday function the exact date we want in the followinf format : (years, months, days, hours, minutes, secondes).

Today we are the 4th February, 2021 and it's around 6PM so we will write (2021, 2, 4, 18, 5, 0) in the jday function.

In [4]:
jd, fr = jday(2021, 2, 4, 18, 5, 0) 

Then we get the error 'e', position vector 'r' & speed vector 'v' in Cartesion coordinates

In [5]:
e, r, v = satellite.sgp4(jd, fr)

In [6]:
print("e : " + str(e) + "\n" + "r : " + str(r) + "\n" + "v : " + str(v))

e : 0
r : (-2137.818424827055, -5365.884343148369, -3589.122249361432)
v : (4.175137064023087, -4.630433520287019, 4.4419129869779415)


# Now, lets process a bunch of TLEs

In [31]:
import pandas as pd

Go back to the previous link : https://celestrak.com/satcat/search-results.php

But this time click on the TLE icon in the first column : "Internal Designator" (also available at this link : https://celestrak.com/satcat/tle.php?INTDES=1998-067)

Right click anywhere on the page an save the file where this code is saved. The file will be saved as tle.txt

In [7]:
TLE_file = "tle.txt"

We then open the file and save the values in three lists :

- L_Name (containing the satellites' names)

- L_1, L_2 (containing respectively the first and second rows of each TLE)

In [60]:
TLEs = open(TLE_file, 'r')

L_Name = []
L_1 = []
L_2 = []

i = 1

for line in TLEs:
    j = i
    if i == 1:
        L_Name.append(line)
        j = 2
    elif i == 2:
        L_1.append(line[:69])
        j = 3
    elif i == 3:
        L_2.append(line[:69])
        j = 1
    i = j

We create a dataframe to gather our data :

In [72]:
dataframe = pd.DataFrame(columns = ['Satellite_name', 'Line_1', 'Line_2', 'Position_vector', 'Speed_vector']) 
dataframe

,Satellite_name,Line_1,Line_2,Position_vector,Speed_vector


We fill in the columns of our dataframe with our data :

In [73]:
dataframe.Satellite_name = L_Name
dataframe.Line_1 = L_1
dataframe.Line_2 = L_2

We pick a date & associate it with the variables jd (Julian Date) and fr (fraction of Julian Date).

Then for each line we compute the position and speed vectors and we put the results in lists :

In [74]:
jd, fr = jday(2021, 2, 4, 18, 5, 0)

L_PosVector = []
L_SpeedVector = []

for i in range(len(dataframe)):
   
    s = dataframe.Line_1[i]
    t = dataframe.Line_2[i]
    satellite = Satrec.twoline2rv(s, t)
    e, r, v = satellite.sgp4(jd, fr)
    L_PosVector.append(r)
    L_SpeedVector.append(v)

Finally, we assign the values of the position and speed vectors to the respectiv columns of our dataset :

In [77]:
dataframe.Position_vector = L_PosVector
dataframe.Speed_vector = L_SpeedVector

In [78]:
dataframe

,Satellite_name,Line_1,Line_2,Position_vector,Speed_vector
0,ISS (ZARYA) \n,1 25544U 98067A 21035.51324206 .00001077 0...,2 25544 51.6455 278.9410 0002184 336.6191 80...,"(-2137.818424827055, -5365.884343148369, -3589...","(4.175137064023087, -4.630433520287019, 4.4419..."
1,KESTREL EYE IIM (KE2M) \n,1 42982U 98067NE 21035.07784222 .00039518 0...,2 42982 51.6323 171.3925 0003428 269.8493 90...,"(3259.2488357546895, 3252.9470579652548, -4877...","(-6.61442207086561, 3.2447383558170495, -2.255..."
2,SIMPL \n,1 42983U 98067NF 21035.43798448 .00008199 0...,2 42983 51.6361 214.0230 0003494 200.7330 159...,"(-6131.33868083971, -2033.3893522594967, -1983...","(-0.05624971777802427, -5.270704914997986, 5.5..."
3,ECAMSAT \n,1 43019U 98067NG 21035.45134322 .00021928 0...,2 43019 51.6389 189.4470 0003522 205.4668 154...,"(-768.1328539999155, 4112.390910933398, -5276....","(-7.59387077374121, -1.1919833818647645, 0.173..."
4,DELLINGR (RBLE) \n,1 43021U 98067NJ 21035.40749557 .00057847 0...,2 43021 51.6297 166.1313 0001881 250.3584 109...,"(6292.3944463409125, -2226.690729427155, 560.5...","(2.0850541370037305, 4.359768960598111, -6.017..."
...,...,...,...,...,...
58,OBJECT RU \n,1 46924U 98067RU 21035.02823493 .00017701 0...,2 46924 51.6414 279.5858 0001095 50.1592 309...,"(4032.2527835041187, 1441.4335591898925, 5253....","(-1.4958491759761905, 7.469229606280735, -0.89..."
59,LEMUR-2-BAXTER-OLIVER \n,1 46925U 98067RV 21034.95410699 .00019862 0...,2 46925 51.6424 279.8471 0001239 34.8523 325...,"(1169.6586754856028, 6279.494280750455, 2279.1...","(-4.610120327018833, 2.839597306018817, -5.432..."
60,LEMUR-2-DJARA \n,1 46926U 98067RW 21034.89644397 .00016585 0...,2 46926 51.6422 280.2113 0000525 33.5776 326...,"(3429.1155136793864, 3406.6405796472113, 4751....","(-2.8251500011349946, 6.6046566019132, -2.6888..."
61,DESCENT \n,1 46927U 98067RX 21035.08025837 .00008521 0...,2 46927 51.6422 279.9325 0001547 4.0934 356...,"(458.7706459146908, 6598.936420263448, 1531.37...","(-4.7760255409536345, 1.6761367883965714, -5.7..."
